In [1]:
import sys
import glob
sys.path.append("../../")

from python.src.train import experiment_tracking_train
from python.src.sat_representations import VCG, LCG, SATRepresentation
from pathlib import Path
NUM_EPOCHS = 20  # 10
f = 0.0000001
alpha = 1
beta = 0
gamma = 0
batch_size = 1
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/GIT_SAT_ML/data/blocksworld"
path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset"
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/GIT_SAT_ML/data/LLL_sample_one"
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/samples_LLL_n80/"
N_STEPS_MOSER = 100
N_RUNS_MOSER = 5
SEED = 0
graph_representation = "LCG"
network_type = "interaction"
# network_definition = get_network_definition(network_type = network_type, graph_representation = graph_representation) #network_definition_interaction_new

MODEL_REGISTRY = Path("../../../mlrun_save")
EXPERIMENT_NAME = "trash"

model_path = False # "../../params_save/" + EXPERIMENT_NAME
img_path = False # model_path + "_plot"

match graph_representation:
    case "LCG":
            rep = LCG
    case "VCG":
            rep = VCG
print(glob.glob(glob.escape(path) + "/*.cnf"))


experiment_tracking_train(
        MODEL_REGISTRY,
        EXPERIMENT_NAME,
        batch_size,
        f,
        alpha,
        beta,
        gamma,
        NUM_EPOCHS,
        N_STEPS_MOSER,
        N_RUNS_MOSER,
        path,
        img_path=img_path,
        model_path=model_path,
        graph_representation=rep,
        network_type=network_type,
    )

['/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_9283351.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_6639307.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_4391198.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_3602064.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_2658295.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_2924554.cnf']


AttributeError: 'list' object has no attribute 'values'